In [1]:
import json

filename = "dev/047_functions/finetune_01/047_func_01_train_oai.jsonl"

with open(filename, "r") as f:
    data = f.readlines()

print(data[0])

{"messages": [{"role": "system", "content": "You are a superintelligent python interpreter. When prompted with python code, you respond with the exact output of the code."}, {"role": "user", "content": "from functions import adarnq, okzfyc\n\nx = -46\n\nprint(17 + okzfyc(x))"}, {"role": "assistant", "content": "-40"}]}



In [3]:
from datasets import load_dataset
train_jsonl_file = "dev/047_functions/finetune_01/047_func_01_train_oai.jsonl"
test_jsonl_file = "dev/047_functions/finetune_01/047_func_01_test_oai.jsonl"

print("Loading datasets...")
dataset = load_dataset(
    "json",
    data_files={
        "train": train_jsonl_file,
        "test": test_jsonl_file,
    },
)
train_dataset = dataset["train"]
test_dataset = dataset["test"]

print(train_dataset[0])
print(test_dataset[0])

Loading datasets...
{'messages': [{'role': 'system', 'content': 'You are a superintelligent python interpreter. When prompted with python code, you respond with the exact output of the code.'}, {'role': 'user', 'content': 'from functions import adarnq, okzfyc\n\nx = -46\n\nprint(17 + okzfyc(x))'}, {'role': 'assistant', 'content': '-40'}]}
{'messages': [{'role': 'system', 'content': 'You are a superintelligent python interpreter. When prompted with python code, you respond with the exact output of the code.'}, {'role': 'user', 'content': 'from functions import psvnji'}, {'role': 'user', 'content': 'Which option correctly describes psvnji?\n\nA) Multiplies the input by 7/4\nB) Returns the input modulo 3\nC) Integer division by 3\nD) Integer division by 4\nE) Multiplies the input by 4\n\nPlease answer with a single uppercase letter corresponding to the correct option.'}, {'role': 'assistant', 'content': 'C'}]}


In [9]:
def to_grpo_prompt_answer(example):
    """
    Convert:
        {'messages': [sys, user, assistant]}
    →   {
          'prompt':  [sys, user],
          'answer':  [assistant_content]
        }
    Notes
    -----
    * The assistant message is assumed to be the **last** element.
    * If you know every assistant output is an integer, you can
      cast it to int here; otherwise leave it as text.
    """
    # keep system + first user
    prompt = example["messages"][:2]

    # assistant output (last message)
    raw_ans = example["messages"][-1]["content"].strip()
    answer = [raw_ans]
    return {"prompt": prompt, "answer": answer}

grpo_train_dataset = train_dataset.map(to_grpo_prompt_answer, remove_columns=["messages"])

print(grpo_train_dataset[0])

Map: 100%|██████████| 96000/96000 [00:03<00:00, 24517.15 examples/s]

{'prompt': [{'content': 'You are a superintelligent python interpreter. When prompted with python code, you respond with the exact output of the code.', 'role': 'system'}, {'content': 'from functions import adarnq, okzfyc\n\nx = -46\n\nprint(17 + okzfyc(x))', 'role': 'user'}], 'answer': ['-40']}


In [10]:
print(grpo_train_dataset[0])

{'prompt': [{'content': 'You are a superintelligent python interpreter. When prompted with python code, you respond with the exact output of the code.', 'role': 'system'}, {'content': 'from functions import adarnq, okzfyc\n\nx = -46\n\nprint(17 + okzfyc(x))', 'role': 'user'}], 'answer': ['-40']}


In [11]:
grpo_test_dataset = test_dataset.map(to_grpo_prompt_answer, remove_columns=["messages"])



Map: 100%|██████████| 1000/1000 [00:00<00:00, 18139.33 examples/s]


In [ ]:
print(grpo_test_dataset[0])
print(type(grpo_test_dataset))
grpo_test_dataset = grpo_test_dataset[:100]
print(grpo_test_dataset[0])

KeyError: 0